## --- Day 19: Beacon Scanner ---

In [1]:
from collections import Counter
from itertools import product
import re
from scipy.spatial.transform import Rotation
from typing import List, Tuple

class Scanner:
    # Must match this many beacons with another Scanner to be considered "overlapping"
    MIN_MATCHING = 12

    def __init__(self, name: str, beacons: list) -> None:
        """
        Initialize from a list of beacons represented as 
        (x, y, z) points relative to the scanner
        """
        self.name = name
        self.beacons = beacons
        self.absolute_position = None      

    def set_origin(self) -> None:
        """Sets this scanner as the origin, 'aligning' it to itself"""
        self.absolute_position = (0, 0, 0)

    def is_aligned(self) -> bool:
        return self.absolute_position is not None

    def try_align_to_scanner(self, other: "Scanner") -> None:
        if not other.is_aligned():
            return

        oriented_beacons, v_diff = self._find_aligned_beacon_set(other)

        if oriented_beacons is None:
            return

        # Save the orientation that aligned (still relative to me)
        self.beacons = oriented_beacons

        # Calculate absolute position relative to origin
        other_x, other_y, other_z = other.absolute_position
        dx, dy, dz = v_diff
        self.absolute_position = (other_x-dx, other_y-dy, other_z-dz)

    def _find_aligned_beacon_set(self, other: "Scanner") -> Tuple[list, Tuple[int]]:
        """
        Rotate all my beacons and compare them to the other Scanner to see
        if at least MIN_MATCHING beacons are overlapping. Returns MY beacons in the
        rotated orientation that aligns with the other Scanner's beacons, or None
        if there is no match
        """
        for rotation in Rotation.create_group(group="O"):
            rotated_beacons = rotation.apply(self.beacons)

            # Compare rotated beacons to the other Scanner's beacons
            differences = []

            for (ax, ay, az), (bx, by, bz) in product(rotated_beacons.astype(int), other.beacons):
                differences.append((int(ax-bx), int(ay-by), int(az-bz)))

            # If there were at least MIN_MATCHING common vectors, can align to this orientation
            v_diff, count = Counter(differences).most_common(1)[0]
            if count >= self.MIN_MATCHING:
                # print(f"Most common diff was {v_diff}")
                # print(f"Intersection of {other.name} and {self.name} is {count}")

                return rotated_beacons.astype(int), v_diff

        return None, None

    def get_absolute_beacons(self) -> List[Tuple[int]]:
        """Return a list of all beacons relative to the origin scanner"""
        absolute_beacons = []
        dx, dy, dz = self.absolute_position
        for beacon in self.beacons:
            x, y, z = beacon
            absolute_beacons.append((x+dx, y+dy, z+dz))

        return absolute_beacons

    @classmethod
    def read_scanners_file(cls, filename: str) -> List["Scanner"]:
        """Load all scanners represented in filename and return a list of Scanner objects"""
        scanners = []
        beacons = []        # Set of beacons for the current Scanner object
        with open(filename) as f:
            for line in f.readlines():
                match = re.search(r"scanner \d+", line)
                if not match and len(line.rstrip()) > 0:
                    # Read beacon for current scanner
                    points = tuple(int(c) for c in line.split(","))
                    beacons.append(points)
                elif match:
                    if len(beacons) > 0:
                        # Create the Scanner for the last set of beacons
                        scanners.append(Scanner(scanner_name, beacons))
                    # Next read a new scanner's list of beacons
                    beacons = []
                    scanner_name = match[0]
        # Add the last scanner read
        scanners.append(Scanner(scanner_name, beacons))

        return scanners


In [2]:
filename = "./inputs/Day19ex.txt"
scanners = Scanner.read_scanners_file(filename)
scanner0 = scanners[0]
scanner1 = scanners[1]

scanner0.set_origin()
scanner1.try_align_to_scanner(scanner0)
intersect_0_1 = set(scanner0.get_absolute_beacons()).intersection(set(scanner1.get_absolute_beacons()))
assert (68, -1246, -43) == scanner1.absolute_position
assert 12 == len(intersect_0_1)

In [3]:
scanner4 = scanners[4]
scanner4.try_align_to_scanner(scanner1)
assert (-20, -1133, 1061) == scanner4.absolute_position

In [4]:
def align_all(filename: str, verbose: bool=True) -> Tuple[list, set]:
    """Load scanners from given file and return list of aligned scanners, tuple of common beacons"""
    unknown_scanners = Scanner.read_scanners_file(filename)

    # Pop the first unknown scanner and mark it as the origin
    curr_scanner = unknown_scanners.pop(0)
    curr_scanner.set_origin()

    # Create a list of known scanners, starting with the origin
    known_scanners = [curr_scanner]

    # Set of all beacons relative to the origin
    all_beacons = set(curr_scanner.beacons)
    beacons_aligned = 0     # for debugging only

    # Process until all unknown scanners are known (assumes they can all be solved!)
    while len(unknown_scanners) > 0:
        curr_scanner = unknown_scanners.pop(0)

        # Try to align this unknown scanner to each known scanner until one aligns
        i = 0
        while not curr_scanner.is_aligned() and i < len(known_scanners):
            curr_scanner.try_align_to_scanner(known_scanners[i])
            i += 1

        if curr_scanner.is_aligned():
            # Move it to the "known" list and add its beacons (relative to origin) to the all_beacons set
            beacons_aligned += len(curr_scanner.beacons)
            all_beacons.update(curr_scanner.get_absolute_beacons())
            known_scanners.append(curr_scanner)
        else:
            # Put it back at the end of the queue and try again later
            unknown_scanners.append(curr_scanner)

    if verbose:
        for s in known_scanners:
            print(f"{s.name} absolute position: {s.absolute_position}")

        print(f"{len(all_beacons)} common beacons out of {beacons_aligned} total")

    return known_scanners, all_beacons

def find_unique_beacons(filename: str, verbose: bool=True) -> int:
    _, unique_beacons = align_all(filename, verbose=verbose)
    
    return unique_beacons

In [5]:
## Example
ex1_filename = "./inputs/Day19ex.txt"
ex1_beacons = find_unique_beacons(filename) 
assert 79 == len(ex1_beacons)

scanner 0 absolute position: (0, 0, 0)
scanner 1 absolute position: (68, -1246, -43)
scanner 3 absolute position: (-92, -2380, -20)
scanner 4 absolute position: (-20, -1133, 1061)
scanner 2 absolute position: (1105, -1205, 1229)
79 common beacons out of 102 total


In [6]:
## Part 1 solution
p1_beacons = find_unique_beacons("./inputs/Day19.txt", verbose=False)
print(len(p1_beacons), "total beacons")

381 total beacons


### --- Part Two ---

What is the largest Manhattan distance between any two scanners?

In the above example, scanners 2 `(1105,-1205,1229)` and 3 `(-92,-2380,-20)` are the largest Manhattan distance apart. In total, they are 1197 + 1175 + 1249 = __3621__ units apart.

In [7]:
def manhattan_dist(point_a: Tuple[int], point_b: Tuple[int]) -> int:
    ax, ay, az = point_a
    bx, by, bz = point_b
    return abs(ax-bx) + abs(ay-by) + abs(az-bz)

def find_max_dist(known_scanners: List["Scanner"]):
    if not known_scanners or len(known_scanners) == 0:
        return None

    max_dist = 0

    while len(known_scanners) > 0:
        curr_scanner = known_scanners.pop(0)
        for scanner in known_scanners:
            curr_dist = manhattan_dist(curr_scanner.absolute_position, scanner.absolute_position)
            # print(f"Dist from {curr_scanner.name} to {scanner.name} is {curr_dist}")
            max_dist = max(max_dist, curr_dist)

    return max_dist

assert 3621 == manhattan_dist((1105, -1205, 1229), (-92, -2380, -20))

In [8]:
# ex1 inputs
known_scanners, _ = align_all("./inputs/Day19ex.txt", verbose=False)
assert 3621 == find_max_dist(known_scanners)

In [9]:
# P2 solution
known_scanners, _ = align_all("./inputs/Day19.txt", verbose=False)
print(f"Aligned {len(known_scanners)} scanners")
print(f"Largest Manhattan distance is {find_max_dist(known_scanners)}")

Aligned 31 scanners
Largest Manhattan distance is 12201
